## Setup

In [1]:
# Intall pyspark and findspark
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()

In [7]:
from pyspark import SparkContext, SparkConf

# Create an Instance of SparkConf
conf = SparkConf()

conf.set("spark.app.name", "Baby_Names_SQL")
conf.set("spark.master", "local[*]")
conf.set("spark.driver.memory", "1g")
conf.set("spark.driver.cores", 1)
conf.set("spark.executor.memory", "4g")
conf.set("spark.executor.cores", 2)

# Initialize Spark using conf
sc = SparkContext(conf=conf)
print(sc)

<SparkContext master=local[*] appName=Baby_Names_SQL>


In [8]:
# import sparksql
from pyspark.sql import SparkSession

spark = SparkSession(sc)

In [9]:
spark

In [19]:
# File path for data source
file_path = "/content/drive/MyDrive/Data Analysis/Simple_Projects/names_data.csv"

# Load data
cols = ["State", "Gender", "Year", "Name", "Births"]
data = spark.read.csv(file_path, header=False, inferSchema=True)


DataFrame[State: string, Gender: string, Year: int, Name: string, Births: int]

In [25]:
data = data.withColumnsRenamed({"_c0": "State", "_c1": "Gender", "_c2": "Year", "_c3": "Name", "_c4": "Births"})

In [26]:
data.printSchema()

root
 |-- State: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Births: integer (nullable = true)



In [27]:
data.show()

+-----+------+----+---------+------+
|State|Gender|Year|     Name|Births|
+-----+------+----+---------+------+
|   AK|     F|1980|  Jessica|   116|
|   AK|     F|1980| Jennifer|   114|
|   AK|     F|1980|    Sarah|    82|
|   AK|     F|1980|   Amanda|    71|
|   AK|     F|1980|  Melissa|    65|
|   AK|     F|1980| Michelle|    61|
|   AK|     F|1980|  Heather|    60|
|   AK|     F|1980|    Amber|    59|
|   AK|     F|1980|  Rebecca|    58|
|   AK|     F|1980|     Erin|    48|
|   AK|     F|1980|Christina|    44|
|   AK|     F|1980|      Amy|    43|
|   AK|     F|1980|Elizabeth|    43|
|   AK|     F|1980|   Angela|    38|
|   AK|     F|1980|     Mary|    38|
|   AK|     F|1980|     Lisa|    36|
|   AK|     F|1980| Kimberly|    35|
|   AK|     F|1980|    Jamie|    34|
|   AK|     F|1980|   Nicole|    32|
|   AK|     F|1980|Stephanie|    32|
+-----+------+----+---------+------+
only showing top 20 rows



In [30]:
data.createTempView("baby_names")

In [45]:
%%time
spark.sql("SELECT State, count(distinct Name) as Count_Names FROM baby_names GROUP BY State").show()

+-----+-----------+
|State|Count_Names|
+-----+-----------+
|   AZ|       4502|
|   SC|       3764|
|   LA|       4754|
|   MN|       3646|
|   NJ|       5670|
|   DC|       1785|
|   OR|       3064|
|   VA|       5184|
|   RI|       1186|
|   KY|       3002|
|   WY|        952|
|   NH|       1180|
|   MI|       6237|
|   NV|       2325|
|   WI|       3763|
|   ID|       1920|
|   CA|      15486|
|   CT|       2536|
|   NE|       2058|
|   MT|       1357|
+-----+-----------+
only showing top 20 rows

CPU times: user 37 ms, sys: 2.92 ms, total: 39.9 ms
Wall time: 7.32 s


In [49]:
# Boys most popular names
query ="""
  SELECT Name, SUM(Births) as num_babies
  FROM baby_names
  WHERE Gender = 'M'
  GROUP BY Name
  ORDER BY num_babies DESC
  Limit 5
  """
spark.sql(query).show()

+-----------+----------+
|       Name|num_babies|
+-----------+----------+
|    Michael|   1376418|
|Christopher|   1118253|
|    Matthew|   1031984|
|     Joshua|    957518|
|     Daniel|    821281|
+-----------+----------+



In [50]:
# Girls most popular names
query = """
  SELECT Name, SUM(Births) as num_babies
  FROM baby_names
  WHERE Gender = 'F'
  GROUP BY Name
  ORDER BY num_babies DESC
  Limit 5
  """
spark.sql(query).show()

+--------+----------+
|    Name|num_babies|
+--------+----------+
| Jessica|    863121|
|  Ashley|    786945|
|Jennifer|    652244|
|   Sarah|    621174|
|  Amanda|    607253|
+--------+----------+



In [ ]:
# Jessica changes in popularity ranking over the years

In [ ]:
# Michael changes in popularity ranking over the years

### Find the names with the biggest jumps in popularity from the first year of the data set to the last year

### For each year, return the 3 most popular girl names and 3 most popular boy names

### For each decade, return the 3 most popular girl names and 3 most popular boy names

In [29]:
# Delete a Temporary View
spark.catalog.dropTempView("baby_names")

# Create or Replace
data.createOrReplaceTempView("baby_names")

True

## Working with RDDs

In [44]:
new_data = sc.parallelize(data.collect(), 4)